# Introduction

In this session, we will attempt a simplified version of the analysis performed in Chu, I.-H. et al. Room-Temperature All-solid-state Rechargeable Sodium-ion Batteries with a Cl-doped Na3PS4 Superionic Conductor. Sci. Rep. 2016, 6, 33733.

# Getting the structure for t-Na3PS4

In [11]:
from pymatgen.ext.matproj import MPRester

mpr = MPRester()  # If this gives you an error, please do mpr = MPRester("your API key") instead.

# Here, we use the high-level interface to the Materials Project (MPRester) to get all entries from 
# the Materials Project with formula Li4GeS4.

entries = mpr.get_entries("Na3PS4", inc_structure=True)
print(len(entries))  # There should be two entries.

# Usually, we want the minimum energy structure
min_entry = min(entries, key=lambda e: e.energy_per_atom)
Na3PS4 = min_entry.structure

# Let us now automatically assign oxidation states to the elements.
Na3PS4.add_oxidation_state_by_guess()

print(Na3PS4)
print("Spacegroup of lowest energy Na3PS4 is %s (%d)" % Na3PS4.get_space_group_info())

2
Full Formula (Na6 P2 S8)
Reduced Formula: Na3PS4
abc   :   6.952000   6.952000   7.075700
angles:  90.000000  90.000000  90.000000
Sites (16)
  #  SP          a        b        c
---  ----  -------  -------  -------
  0  Na+   0        0.5      0.42066
  1  Na+   0.5      0        0.57934
  2  Na+   0.5      0        0.07934
  3  Na+   0        0.5      0.92066
  4  Na+   0        0        0
  5  Na+   0.5      0.5      0.5
  6  P5+   0        0        0.5
  7  P5+   0.5      0.5      0
  8  S2-   0.31244  0.34755  0.16502
  9  S2-   0.34755  0.68756  0.83498
 10  S2-   0.65245  0.31244  0.83498
 11  S2-   0.81244  0.15245  0.33498
 12  S2-   0.18756  0.84755  0.33498
 13  S2-   0.68756  0.65245  0.16502
 14  S2-   0.15245  0.18756  0.66502
 15  S2-   0.84755  0.81244  0.66502
Spacegroup of lowest energy Na3PS4 is P-42_1c (114)


# Creating Cl-doped Na3PS4

In [24]:
Na3PS4Cl = Na3PS4.copy()

# Note that in reality, a supercell has to be generated to achieve the low Cl concentrations in experiments.
# The code to generate a 2x2x2 supercell is as follows:
# Na3PS4Cl.make_supercell([2, 2, 2])

# Here, we will only perform analysis within a 1x1x1 conventional cell in the interests of time.

Na3PS4Cl["S2-"] = {"S2-": 7/8, "Cl-": 1/8}
Na3PS4Cl["Na+"] = {"Na+": 5/6}
print("Overall charge of %s is %f" % (Na3PS4Cl.formula, Na3PS4Cl.charge))

# Generates a crystallographic information format file that can be viewed in most 
# crystal visualization software.
Na3PS4Cl.to(filename="Na3PS4Cl.cif")  

Overall charge of Na5 P2 S7 Cl1 is 0.000000


To perform calculations on Cl-doped Na3PS4, we need to generate ordered crystals from the disordered crystal we just created. We will make use of Pymatgen's transformations package, which are basically standardized series of operations on structures to generate new structures.

In [26]:
from pymatgen.transformations.standard_transformations import OrderDisorderedStructureTransformation

trans = OrderDisorderedStructureTransformation()
ordered_Na3PS4Cl = trans.apply_transformation(Na3PS4Cl, return_ranked_list=100)
print("# of structures generated = %s" % len(ordered_Na3PS4Cl))

# of structures generated = 48


Note that the OrderedDisorderedStructureTransformation does not take into account symmetry. We will use Pymatgen's powerful StructureMatcher to obtain only symmetrically distinct structures.

In [29]:
from pymatgen.analysis.structure_matcher import StructureMatcher

matcher = StructureMatcher()
groups = matcher.group_structures([d["structure"] for d in ordered_Na3PS4Cl])
distinct_Na3PS4Cl = [g[0] for g in groups]
print(len(distinct_Na3PS4Cl))

2


# Generating input files for calculations

In [31]:
from pymatgen.io.vasp.sets import MPRelaxSet, batch_write_input

# batch_write_input(distinct_Na3PS4Cl, include_cif=True)

# Retreiving the results of a calculation

In [ ]:
from pymatgen.io.vasp import Vasprun

vasprun = Vasprun("vasprun.Li4SnS4.xml")
print(vasprun.final_structure)
print(vasprun.final_energy)
entry = vasprun.get_computed_entry()